In [1]:
# Load files
import pandas as pd

ratings = pd.read_csv('C:/Users/JHON G. BOTELLO/OneDrive - Old Dominion University/PHD/Courses/Spring 2024/Web Science/Web-Science/HW6-Recommender Systems/Data/ml-100k/ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
print("Ratings Data:")
print(ratings.head())

print ("--------------------------------")

movies = pd.read_csv('C:/Users/JHON G. BOTELLO/OneDrive - Old Dominion University/PHD/Courses/Spring 2024/Web Science/Web-Science/HW6-Recommender Systems/Data/ml-100k/ml-100k/u.item', sep='|', header=None, encoding='latin-1', 
                     names=['item_id', 'title', 'release_date', 'video_release_date', 
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 
                            "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 
                            'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'])
print("\nMovies Data:")
print(movies.head())

print ("--------------------------------")

users = pd.read_csv('C:/Users/JHON G. BOTELLO/OneDrive - Old Dominion University/PHD/Courses/Spring 2024/Web Science/Web-Science/HW6-Recommender Systems/Data/ml-100k/ml-100k/u.user', sep='|', header=None, 
                    names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])
print("\nUsers Data:")
print(users.head())

Ratings Data:
   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596
--------------------------------

Movies Data:
   item_id              title release_date  video_release_date  \
0        1   Toy Story (1995)  01-Jan-1995                 NaN   
1        2   GoldenEye (1995)  01-Jan-1995                 NaN   
2        3  Four Rooms (1995)  01-Jan-1995                 NaN   
3        4  Get Shorty (1995)  01-Jan-1995                 NaN   
4        5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb_URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imd

### Q1: Find 3 users who are closest to you in terms of age, gender, and occupation. 

In [2]:
print(users['occupation'].unique())

['technician' 'other' 'writer' 'executive' 'administrator' 'student'
 'lawyer' 'educator' 'scientist' 'entertainment' 'programmer' 'librarian'
 'homemaker' 'artist' 'engineer' 'marketing' 'none' 'healthcare' 'retired'
 'salesman' 'doctor']


In [3]:
# My demographic details
jhon_age = 25
jhon_gender = 'M'
jhon_occupation = 'student'

closest_users = users[(users['age'] == jhon_age) & (users['gender'] == jhon_gender) & (users['occupation'] == jhon_occupation)]
closest_users = closest_users.head(3)

print("The 3 users closest to me are:")
print(closest_users)

The 3 users closest to me are:
     user_id  age gender occupation zip_code
152      153   25      M    student    60641
153      154   25      M    student    53703
247      248   25      M    student    37235


In [4]:
# For each of those 3 users:
# * *Q: What are their top 3 (favorite) films?*
# * *Q: What are their bottom 3 (least favorite) films?*

def get_top_and_bottom_movies(user_id, n=3):
    user_ratings = ratings[ratings['user_id'] == user_id]
    merged = user_ratings.merge(movies[['item_id', 'title']], on='item_id')

    top_movies = merged.nlargest(n, 'rating')[['title', 'rating']]
    bottom_movies = merged.nsmallest(n, 'rating')[['title', 'rating']]

    return top_movies, bottom_movies

for _, user in closest_users.iterrows():
    user_id = user['user_id']
    print(f"\nUser {user_id}:")
    top_movies, bottom_movies = get_top_and_bottom_movies(user_id)
    
    print("Top 3 Movies:")
    print(top_movies)
    
    
    print("Bottom 3 Movies:")
    print(bottom_movies)


User 153:
Top 3 Movies:
                               title  rating
2   Shawshank Redemption, The (1994)       5
6                Pulp Fiction (1994)       5
13                    Contact (1997)       5
Bottom 3 Movies:
                             title  rating
0   Raiders of the Lost Ark (1981)       1
11                Star Wars (1977)       1
15       Return of the Jedi (1983)       1

User 154:


Top 3 Movies:
                title  rating
5       Brazil (1985)       5
12  GoodFellas (1990)       5
13   Star Wars (1977)       5
Bottom 3 Movies:
                              title  rating
2                      Evita (1996)       2
36              Lost Highway (1997)       2
41  Star Trek: First Contact (1996)       2

User 248:
Top 3 Movies:
                   title  rating
2       Rock, The (1996)       5
4   Trainspotting (1996)       5
6  Godfather, The (1972)       5
Bottom 3 Movies:
                  title  rating
38  Forrest Gump (1994)       1
42          Emma (1996)       1
55  Dante's Peak (1997)       1


### Q2

Based on the ratings that users have given to the movies, answer the following questions:

* *Q: Which 5 users are most correlated to the* substitute you *(i.e., which 5 users rate movies most similarly to the* substitute you?)
* *Q: Which 5 users are least correlated (i.e., negative correlation)?*
* *Q: Explain the general operation of any functions you use from recommendations.py.*

In [5]:
# Pearson correlation function from recomedation.py

from math import sqrt

def sim_pearson(prefs, p1, p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2.
    '''

    # Get the list of mutually rated items
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1
    
    # If they are no ratings in common, return 0
    if len(si) == 0:
        return 0
    
    # Sum calculations
    n = len(si)
    
    # Sums of all the preferences
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])
    
    # Sums of the squares
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])
    
    # Sum of the products
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    
    # Calculate r (Pearson score)
    num = pSum - sum1 * sum2 / n
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0:
        return 0
    r = num / den
    return r


In [6]:
prefs = {}
for _, row in ratings.iterrows():
    prefs.setdefault(row['user_id'], {})
    prefs[row['user_id']][row['item_id']] = row['rating']

substitute_jhon = 153
correlations = []

for user_id in prefs:
    if user_id != substitute_jhon:
        corr = sim_pearson(prefs, substitute_jhon, user_id)
        correlations.append((user_id, corr))

correlations.sort(key=lambda x: x[1], reverse=True)

most_correlated = correlations[:5]
print("Top 5 Most Correlated Users:")
for user, corr in most_correlated:
    print(f"User {user}: Pearson Correlation = {corr:.3f}")

least_correlated = correlations[-5:]
print("\nTop 5 Least Correlated Users:")
for user, corr in least_correlated:
    print(f"User {user}: Pearson Correlation = {corr:.3f}")

Top 5 Most Correlated Users:
User 163: Pearson Correlation = 1.000
User 469: Pearson Correlation = 1.000
User 277: Pearson Correlation = 1.000
User 340: Pearson Correlation = 1.000
User 284: Pearson Correlation = 1.000

Top 5 Least Correlated Users:
User 939: Pearson Correlation = -1.000
User 257: Pearson Correlation = -1.000
User 258: Pearson Correlation = -1.000
User 32: Pearson Correlation = -1.000
User 231: Pearson Correlation = -1.000


### Q3: Compute ratings for all the films that the *substitute you* has not seen and answer the following questions:

**Q: What are the top 5 recommendations for films that the *substitute you* should see.?**

**Q: What are the bottom 5 recommendations (i.e., films the *substitute you* is almost certain to hate)?**

In [7]:
#getRecomendations form recomendation.py
def getRecommendations(prefs, person, similarity=sim_pearson):
    '''
    Gets recommendations for a person by using a weighted average
    of every other user's rankings
    '''

    totals = {}
    simSums = {}
    for other in prefs:
    # Don't compare me to myself
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        # Ignore scores of zero or lower
        if sim <= 0:
            continue
        for item in prefs[other]:
            # Only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item] == 0:
                # Similarity * Score
                totals.setdefault(item, 0)
                # The final score is calculated by multiplying each item by the
                #   similarity and adding these products together
                totals[item] += prefs[other][item] * sim
                # Sum of similarities
                simSums.setdefault(item, 0)
                simSums[item] += sim
    # Create the normalized list
    rankings = [(total / simSums[item], item) for (item, total) in
                totals.items()]
    # Return the sorted list
    rankings.sort()
    rankings.reverse()
    return rankings

In [9]:
substitute_user = 153
recommendations = getRecommendations(prefs, substitute_user, similarity=sim_pearson)

# Map item_id to movie titles and display the top and bottom recommendations
recommendation_list = [(rating, movies.loc[movies['item_id'] == item_id, 'title'].iloc[0]) 
                       for rating, item_id in recommendations]

# Top 5 Recommendations
print("Top 5 Movie Recommendations:")
for predicted_rating, title in recommendation_list[:5]:
    print(f"{title}: Predicted Rating = {predicted_rating:.2f}")

# Bottom 5 Recommendations
print("\nBottom 5 Movie Recommendations:")
for predicted_rating, title in recommendation_list[-5:]:
    print(f"{title}: Predicted Rating = {predicted_rating:.2f}")

Top 5 Movie Recommendations:
Little City (1998): Predicted Rating = 5.00
Entertaining Angels: The Dorothy Day Story (1996): Predicted Rating = 5.00
Aparajito (1956): Predicted Rating = 5.00
Aiqing wansui (1994): Predicted Rating = 5.00
World of Apu, The (Apur Sansar) (1959): Predicted Rating = 5.00

Bottom 5 Movie Recommendations:
Amityville 3-D (1983): Predicted Rating = 1.00
Amityville 1992: It's About Time (1992): Predicted Rating = 1.00
3 Ninjas: High Noon At Mega Mountain (1998): Predicted Rating = 1.00
Turbo: A Power Rangers Movie (1997): Predicted Rating = 1.00
Theodore Rex (1995): Predicted Rating = 1.00


### Q4: Choose your (the real you, not the *substitute you*) favorite and least favorite film from the data. 

* *Q: What are the top 5 most correlated films to your favorite film?  Bottom 5 least correlated?*
* *Q: What are the top 5 most correlated films to your least favorite film?  Bottom 5 least correlated?*
* *Q: Based on your knowledge of the resulting films, do you agree with the results?*  In other words, do you personally like/dislike the resulting films? 
* *Q: Explain the general operation of any functions you use from recommendations.py.*

In [11]:
favorite_movie_id = 82  # Jurassic Park (1993) 
least_favorite_movie_id = 79  # Fugitive (1993)

In [ ]:
#Funtion from recomendation.py
def topMatches(
    prefs,
    person,
    n=5,
    similarity=sim_pearson,
):
    '''
    Returns the best matches for person from the prefs dictionary. 
    Number of results and similarity function are optional params.
    '''

    scores = [(similarity(prefs, person, other), other) for other in prefs
              if other != person]
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [13]:
#Funtion from recomendation.py
def transformPrefs(prefs):
    '''
    Transform the recommendations into a mapping where persons are described
    with interest scores for a given title e.g. {title: person} instead of
    {person: title}.
    '''

    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})
            # Flip item and person
            result[item][person] = prefs[person][item]
    return result

In [14]:
movies = transformPrefs(prefs)

In [ ]:
favorite_film_id = 82 
top_similar = topMatches(movies, favorite_film_id, n=5)

print("Top 5 Most Correlated Films:")
for score, movie_id in top_similar:
    print(f"Movie ID: {movie_id}, Similarity Score: {score}")

Top 5 Most Correlated Films:
Movie ID: 839, Similarity Score: 1.0000000000000004
Movie ID: 1629, Similarity Score: 1.0
Movie ID: 1607, Similarity Score: 1.0
Movie ID: 1605, Similarity Score: 1.0
Movie ID: 1554, Similarity Score: 1.0


In [22]:
def bottomMatches(prefs, person, n=5, similarity=sim_pearson):
    '''
    Returns the least correlated matches for a given film.
    '''
    scores = [(similarity(prefs, person, other), other) for other in prefs if other != person]
    scores.sort()  # Sort in ascending order for least correlation
    return scores[:n]

least_similar = bottomMatches(movies, favorite_film_id, n=5)

print("Bottom 5 Least Correlated Films:")
for score, movie_id in least_similar:
    print(f"Movie ID: {movie_id}, Similarity Score: {score}")


Bottom 5 Least Correlated Films:
Movie ID: 1524, Similarity Score: -1.0000000000000018
Movie ID: 360, Similarity Score: -1.0
Movie ID: 643, Similarity Score: -1.0
Movie ID: 666, Similarity Score: -1.0
Movie ID: 850, Similarity Score: -1.0
